## Part 2 - TF-IDF with LSTM

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import SGDClassifier
from gensim.models import Word2Vec

from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Fatih\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fatih\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Fatih\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Fatih\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# Load data
df = pd.read_csv('IMDB_Dataset_Preprocessed.csv') 
df.head()

,Unnamed: 0,review,sentiment,cleaned_review,sentiment_numeric,tokens
0,0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching oz episode you...,1,"['one', 'reviewer', 'mentioned', 'watching', '..."
1,1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...,1,"['wonderful', 'little', 'production', 'filming..."
2,2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,1,"['thought', 'wonderful', 'way', 'spend', 'time..."
3,3,Basically there's a family where a little boy ...,negative,basically there family little boy jake think t...,0,"['basically', 'there', 'family', 'little', 'bo..."
4,4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...,1,"['petter', 'matteis', 'love', 'time', 'money',..."


## LSTM related classes and methods

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix


# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# 1. Define Custom Dataset Class
class NumpyDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        x = self.features[idx]
        y = self.targets[idx]
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return x, y

# 2. Define LSTM Model Class
class LSTMNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x.unsqueeze(1), (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# 3. Training Loop
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device).view(-1, 1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# 4. Create Test Function
def test_model(model, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device).view(-1, 1)
            outputs = model(inputs)
            all_preds.append(outputs.cpu().numpy())
            all_targets.append(targets.cpu().numpy())
    all_preds = np.concatenate(all_preds).flatten().round()
    all_targets = np.concatenate(all_targets).flatten()
    return all_targets, all_preds

# Define LSTM Training Parameters
# input_size = 5000  # Input size
hidden_size = 128  # Example hidden layer size
num_layers = 2 # Number of LSTM layers
output_size = 1 # Output size (single scalar value)
batch_size = 32 # Define Batch Size
num_epochs = 10 # Number of epochs

c:\Users\Fatih\miniconda3\envs\fth_cuda_3_8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [4]:
# Function to print evaluation metrics
def evaluate_model(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")

In [5]:
# Tokenize cleaned reviews for Word2Vec model
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

# Apply preprocessing to all reviews
df['tokens'] = df['cleaned_review'].apply(tokenize_text)

## Tuning Window Size

In [6]:
input_size = 100  # Input size (Word2Vec dimensions)   

### Window Size 3

In [ ]:
# Create Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=3, min_count=5, workers=4, epochs=10)
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in df['tokens']])
print("Word2Vec Shape using window size 3:", X_word2vec.shape)

# Convert to numpy array
X_word2vec = np.asarray(X_word2vec)

scaler = StandardScaler()
model = scaler.fit(X_word2vec)
X_word2vec = model.transform(X_word2vec)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, df['sentiment_numeric'].values, test_size=0.5, random_state=42)


In [ ]:
train_dataset = NumpyDataset(X_train, y_train)
test_dataset = NumpyDataset(X_test, y_test)

In [ ]:
# Prepare DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
# Instantiate Model, Define Loss and Optimizer
model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Run Training Loops
train_model(model, train_loader, criterion, optimizer, num_epochs)


In [ ]:
# Run Test
y_test, y_pre = test_model(model, test_loader)
print("LSTM with word2vec using window size 3:")
evaluate_model(y_test, y_pre)

### Window Size 5

In [ ]:
# Create Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=5, workers=4, epochs=10)
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in df['tokens']])
print("Word2Vec Shape using window size 5:", X_word2vec.shape)

# Convert to numpy array
X_word2vec = np.asarray(X_word2vec)

scaler = StandardScaler()
model = scaler.fit(X_word2vec)
X_word2vec = model.transform(X_word2vec)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, df['sentiment_numeric'].values, test_size=0.5, random_state=42)


In [ ]:
train_dataset = NumpyDataset(X_train, y_train)
test_dataset = NumpyDataset(X_test, y_test)

In [ ]:
# Prepare DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
# Instantiate Model, Define Loss and Optimizer
model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Run Training Loops
train_model(model, train_loader, criterion, optimizer, num_epochs)

In [ ]:
# Run Test
y_test, y_pre = test_model(model, test_loader)
print("LSTM with word2vec using window size 5:")
evaluate_model(y_test, y_pre)

### Window Size 7

In [ ]:
# Create Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=7, min_count=5, workers=4, epochs=10)
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in df['tokens']])
print("Word2Vec Shape using window size 7:", X_word2vec.shape)

# Convert to numpy array
X_word2vec = np.asarray(X_word2vec)

scaler = StandardScaler()
model = scaler.fit(X_word2vec)
X_word2vec = model.transform(X_word2vec)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, df['sentiment_numeric'].values, test_size=0.5, random_state=42)


In [ ]:
train_dataset = NumpyDataset(X_train, y_train)
test_dataset = NumpyDataset(X_test, y_test)

In [ ]:
# Prepare DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
# Instantiate Model, Define Loss and Optimizer
model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Run Training Loops
train_model(model, train_loader, criterion, optimizer, num_epochs)

In [ ]:
# Run Test
y_test, y_pre = test_model(model, test_loader)
print("LSTM with word2vec using window size 7:")
evaluate_model(y_test, y_pre)

## Tuning Embedding Dimension

### Embedding Dimension 50

In [ ]:
input_size = 50 # Input size (Word2Vec dimensions)   

In [ ]:
# Create Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=50, window=5, min_count=5, workers=4, epochs=10)
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(50)], axis=0) for words in df['tokens']])
print("Word2Vec Shape using embedding dimension 50:", X_word2vec.shape)

# Convert to numpy array
X_word2vec = np.asarray(X_word2vec)

scaler = StandardScaler()
model = scaler.fit(X_word2vec)
X_word2vec = model.transform(X_word2vec)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, df['sentiment_numeric'].values, test_size=0.5, random_state=42)


In [ ]:
train_dataset = NumpyDataset(X_train, y_train)
test_dataset = NumpyDataset(X_test, y_test)

In [ ]:
# Prepare DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
# Instantiate Model, Define Loss and Optimizer
model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Run Training Loops
train_model(model, train_loader, criterion, optimizer, num_epochs)


In [ ]:
# Run Test
y_test, y_pre = test_model(model, test_loader)
print("LSTM with word2vec using embedding dimension 50:")
evaluate_model(y_test, y_pre)

### Embedding Dimension 100

In [ ]:
input_size = 100 # Input size (Word2Vec dimensions)   

In [ ]:
# Create Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=5, workers=4, epochs=10)
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in df['tokens']])
print("Word2Vec Shape using embedding dimension 100:", X_word2vec.shape)

# Convert to numpy array
X_word2vec = np.asarray(X_word2vec)

scaler = StandardScaler()
model = scaler.fit(X_word2vec)
X_word2vec = model.transform(X_word2vec)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, df['sentiment_numeric'].values, test_size=0.5, random_state=42)


In [ ]:
train_dataset = NumpyDataset(X_train, y_train)
test_dataset = NumpyDataset(X_test, y_test)

In [ ]:
# Prepare DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
# Instantiate Model, Define Loss and Optimizer
model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Run Training Loops
train_model(model, train_loader, criterion, optimizer, num_epochs)

In [ ]:
# Run Test
y_test, y_pre = test_model(model, test_loader)
print("LSTM with word2vec using embedding dimension 100:")
evaluate_model(y_test, y_pre)

### Embedding Dimension 200

In [ ]:
input_size = 200 # Input size (Word2Vec dimensions)   

In [ ]:
# Create Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=200, window=5, min_count=5, workers=4, epochs=10)
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(200)], axis=0) for words in df['tokens']])
print("Word2Vec Shape using embedding dimension 200:", X_word2vec.shape)

# Convert to numpy array
X_word2vec = np.asarray(X_word2vec)

scaler = StandardScaler()
model = scaler.fit(X_word2vec)
X_word2vec = model.transform(X_word2vec)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, df['sentiment_numeric'].values, test_size=0.5, random_state=42)


In [ ]:
train_dataset = NumpyDataset(X_train, y_train)
test_dataset = NumpyDataset(X_test, y_test)

In [ ]:
# Prepare DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
# Instantiate Model, Define Loss and Optimizer
model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Run Training Loops
train_model(model, train_loader, criterion, optimizer, num_epochs)

In [ ]:
# Run Test
y_test, y_pre = test_model(model, test_loader)
print("LSTM with word2vec using embedding dimension 200:")
evaluate_model(y_test, y_pre)

## Tuning Epoch Number

In [ ]:
input_size = 100

### Epoch Number 5

In [ ]:
# Create Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=5, workers=4, epochs=5)
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in df['tokens']])
print("Word2Vec Shape using epoch number 5:", X_word2vec.shape)

# Convert to numpy array
X_word2vec = np.asarray(X_word2vec)

scaler = StandardScaler()
model = scaler.fit(X_word2vec)
X_word2vec = model.transform(X_word2vec)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, df['sentiment_numeric'].values, test_size=0.5, random_state=42)


In [ ]:
train_dataset = NumpyDataset(X_train, y_train)
test_dataset = NumpyDataset(X_test, y_test)

In [ ]:
# Prepare DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
# Instantiate Model, Define Loss and Optimizer
model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Run Training Loops
train_model(model, train_loader, criterion, optimizer, num_epochs)

In [ ]:
# Run Test
y_test, y_pre = test_model(model, test_loader)
print("LSTM with word2vec using epoch number 5:")
evaluate_model(y_test, y_pre)

### Epoch Number 10

In [ ]:
# Create Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=5, workers=4, epochs=10)
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in df['tokens']])
print("Word2Vec Shape using epoch number 10:", X_word2vec.shape)

# Convert to numpy array
X_word2vec = np.asarray(X_word2vec)

scaler = StandardScaler()
model = scaler.fit(X_word2vec)
X_word2vec = model.transform(X_word2vec)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, df['sentiment_numeric'].values, test_size=0.5, random_state=42)


In [ ]:
train_dataset = NumpyDataset(X_train, y_train)
test_dataset = NumpyDataset(X_test, y_test)

In [ ]:
# Prepare DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
# Instantiate Model, Define Loss and Optimizer
model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Run Training Loops
train_model(model, train_loader, criterion, optimizer, num_epochs)

In [ ]:
# Run Test
y_test, y_pre = test_model(model, test_loader)
print("LSTM with word2vec using epoch number 10:")
evaluate_model(y_test, y_pre)

### Epoch Number 20

In [ ]:
# Create Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=5, workers=4, epochs=20)
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in words if word in word2vec_model.wv] or [np.zeros(100)], axis=0) for words in df['tokens']])
print("Word2Vec Shape using epoch number 20:", X_word2vec.shape)

# Convert to numpy array
X_word2vec = np.asarray(X_word2vec)

scaler = StandardScaler()
model = scaler.fit(X_word2vec)
X_word2vec = model.transform(X_word2vec)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, df['sentiment_numeric'].values, test_size=0.5, random_state=42)


In [ ]:
train_dataset = NumpyDataset(X_train, y_train)
test_dataset = NumpyDataset(X_test, y_test)

In [ ]:
# Prepare DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
# Instantiate Model, Define Loss and Optimizer
model = LSTMNetwork(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Run Training Loops
train_model(model, train_loader, criterion, optimizer, num_epochs)

In [ ]:
# Run Test
y_test, y_pre = test_model(model, test_loader)
print("LSTM with word2vec using epoch number 20:")
evaluate_model(y_test, y_pre)